# Saving and loading populations

Another feature Brush implements is the ability to save and load entire populations.
We use JSON notation to store the population into a file that is human readable. The same way, we can feed an estimator a previous population file to serve as starting point for the evolution.

In this notebook, we will walk through how to use the `save_population` and `load_population` parameters. 

We start by getting a sample dataset and splitting it into `X` and `y`:

In [1]:
import pandas as pd
from pybrush import BrushRegressor

# load data
df = pd.read_csv('../examples/datasets/d_enc.csv')
X = df.drop(columns='label')
y = df['label']

To save the population after finishing the evolution, you nee to set `save_population` parameter to a value different than an empty string. Then, the final population is going to be stored in that specific file.

In this example, we create a temporary file.

In [2]:
import pickle
import os, tempfile

pop_file = os.path.join(tempfile.mkdtemp(), 'population.json')

# set verbosity==2 to see the full report
est = BrushRegressor(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    max_gens=10,
    objectives=["scorer", "complexity"],
    save_population=pop_file,
    use_arch=True, # Only the pareto front of last gen will be stored in archive
    verbosity=2
)

est.fit(X,y)
y_pred = est.predict(X)
print('score:', est.score(X,y))

Generation 1/10 [//////                                            ]
Train Loss (Med): 6.88365 (60.79966)
Val Loss (Med): 6.88365 (60.79966)
Median Size (Max): 16 (104)
Median complexity (Max): 808 (1957202248)
Time (s): 1.09453

Generation 2/10 [///////////                                       ]
Train Loss (Med): 5.92244 (15.00702)
Val Loss (Med): 5.92244 (15.00702)
Median Size (Max): 41 (104)
Median complexity (Max): 808 (759781448)
Time (s): 1.61749

Generation 3/10 [////////////////                                  ]
Train Loss (Med): 5.82386 (9.94906)
Val Loss (Med): 5.82386 (9.94906)
Median Size (Max): 55 (99)
Median complexity (Max): 67987852 (979195976)
Time (s): 2.05281

Generation 4/10 [/////////////////////                             ]
Train Loss (Med): 5.78817 (7.14350)
Val Loss (Med): 5.78817 (7.14350)
Median Size (Max): 72 (99)
Median complexity (Max): 190343888 (979195976)
Time (s): 2.68082

Generation 5/10 [//////////////////////////                        ]
Train Los

Loading a previous population is done providing `load_population` a string value corresponding to a JSON file generated by Brush. In our case, we will use the same file from the previous code block.

After loading the population, we run the evolution for 10 more generations, and we can see that the first generation started from the previous population. This means that the population was successfully saved and loaded.

In [3]:
est = BrushRegressor(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    load_population=pop_file,
    max_gens=10,
    verbosity=1
)

est.fit(X,y)
y_pred = est.predict(X)
print('score:', est.score(X,y))

Loaded population from /tmp/tmpwjowtmfv/population.json of size = 200
Completed 100% [====================]
saving final population as archive...
score: 0.9679935651843212


There is a convenient way of accessing individuals on the population: just use the index of the individual on the `est.population_` list.

In [4]:
# it will contain all individuals, differently than the archive
print("population size:", len(est.population_))
print("archive size   :", len(est.archive_))

est.population_[0]

population size: 100
archive size   : 21


{'fitness': {'complexity': 2,
  'crowding_dist': 0.0,
  'dcounter': 0,
  'depth': 1,
  'dominated': [96, 98],
  'linear_complexity': 2,
  'loss': 90.3851318359375,
  'loss_v': 90.3851318359375,
  'rank': 1,
  'size': 1,
  'values': [90.3851318359375, 1.0],
  'weights': [-1.0, -1.0],
  'wvalues': [-90.3851318359375, -1.0]},
 'id': 0,
 'is_fitted_': False,
 'objectives': ['mse', 'size'],
 'parent_id': [],
 'program': {'Tree': [{'W': 24.585369110107422,
    'arg_types': [],
    'center_op': True,
    'feature': 'Constant',
    'fixed': False,
    'is_weighted': True,
    'name': 'Constant',
    'node_type': 'Constant',
    'prob_change': 0.6167147755622864,
    'ret_type': 'ArrayF',
    'sig_dual_hash': 509529941281334733,
    'sig_hash': 17717457037689164349}],
  'is_fitted_': True},
 'variation': 'born'}

you can convert the json representation back to an fully functional individual by wrapping it in the individual class. It is important that the type of individual (i.e. classification, regression) is the same.

Differently from the archive (which is sorted by complexity), the individuals in the population have no specific order. So individual 5 may or may not be more complex than individual 10, for example.

In [5]:
from pybrush import individual

loaded_from_population = individual.RegressorIndividual.from_json(est.population_[2])
print(loaded_from_population.get_model("tree"))

24.59


## Saving just the archive

In case you want to use another expression rather than the final `best_estimator_`, brush provides the archive option.

The archive is just the pareto front from the population. You can use `predict_archive` (and `predict_proba_archive` if using a `BrushClassifier`) to call the prediction methods for the entire archive, instead of the selected best individual.

But first, you need to enable this option with `use_arch=True`. When set to False, it will store the entire final population

In [6]:
est = BrushRegressor(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    load_population=pop_file,
    use_arch=True,
    max_gens=10,
    verbosity=1
)

est.fit(X,y)

# accessing first expression from the archive. It is serialized as a dict
print(est.archive_[0]['fitness'])

Loaded population from /tmp/tmpwjowtmfv/population.json of size = 200
Completed 100% [====================]
{'complexity': 2822449224, 'crowding_dist': 0.0, 'dcounter': 0, 'depth': 9, 'dominated': [], 'linear_complexity': 231, 'loss': 4.055662631988525, 'loss_v': 4.055662631988525, 'rank': 1, 'size': 70, 'values': [4.055662631988525, 70.0], 'weights': [-1.0, -1.0], 'wvalues': [-4.055662631988525, -70.0]}


You can open the serialized file and change individuals' programs manually.

This also allow us to have checkpoints in the execution.

## Using population files with classification

To give another example, we do a two-step fit in the cells below.

First, we run the evolution and save the population to a file; then, we load it and keep evolving the individuals.

What is different though is that the first run is optimizing `scorer` and `complexity`, and the second run is optimizing `average_precision_score` and `linear_complexity`.

In [7]:
from pybrush import BrushClassifier

# load data
df = pd.read_csv('../examples/datasets/d_analcatdata_aids.csv')
X = df.drop(columns='target')
y = df['target']

pop_file = os.path.join(tempfile.mkdtemp(), 'population.json')

est = BrushClassifier(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    max_gens=10,
    objectives=["scorer", "complexity"],
    scorer="log",
    save_population=pop_file,
    pop_size=200,
    verbosity=2
)

est.fit(X,y)

print("Best model:", est.best_estimator_.get_model())
print('score:', est.score(X,y))

Generation 1/10 [//////                                            ]
Train Loss (Med): 0.50090 (0.69315)
Val Loss (Med): 0.50090 (0.69315)
Median Size (Max): 9 (78)
Median complexity (Max): 992 (2081414560)
Time (s): 0.22601

Generation 2/10 [///////////                                       ]
Train Loss (Med): 0.41101 (0.68320)
Val Loss (Med): 0.41101 (0.68320)
Median Size (Max): 19 (93)
Median complexity (Max): 128 (2065642976)
Time (s): 0.45281

Generation 3/10 [////////////////                                  ]
Train Loss (Med): 0.41101 (0.66391)
Val Loss (Med): 0.41101 (0.66391)
Median Size (Max): 24 (93)
Median complexity (Max): 128 (2065642976)
Time (s): 0.67236

Generation 4/10 [/////////////////////                             ]
Train Loss (Med): 0.41101 (0.56053)
Val Loss (Med): 0.41101 (0.56053)
Median Size (Max): 36 (93)
Median complexity (Max): 18570656 (2065642976)
Time (s): 0.87288

Generation 5/10 [//////////////////////////                        ]
Train Loss (Med): 0

In [8]:
from sklearn.metrics import accuracy_score

accuracy_score(y, est.predict(X))

0.86

In [9]:
est = BrushClassifier(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    load_population=pop_file,
    objectives=["scorer", "complexity"],
    scorer="average_precision_score",
    max_gens=10,
    validation_size=0.0,
    pop_size=200, # make sure this is the same as loaded pop
    use_arch=True,
    verbosity=1
)

est.fit(X,y)

print("Best model:", est.best_estimator_.get_model())
print('score:', est.score(X,y))

Loaded population from /tmp/tmp1quqba5t/population.json of size = 400
Completed 100% [====================]
Best model: Logistic(Sum(33.01,-43.26*Cos(If(AIDS>15890.50,1.00,Sin(0.99*Add(Sum(-0.38,1.54*Sin(1.00*AIDS)),2.57*Cos(2.09*Cos(0.40*Mul(0.41*AIDS,0.41*AIDS)))))))))
score: 0.94


We can see the fitness object, and that the scorer now matches the average precision score metric:

In [10]:
# Fitness is (scorer, linear complexity)
print(est.best_estimator_.fitness)

Fitness(0.895947 876211584.000000 )


In [11]:
from sklearn.metrics import average_precision_score

# takes y_true as first argument, and y_pred as second argument.
average_precision_score(y, est.predict_proba(X)[:, 1]) #, average='weighted')

0.9576863733102123

## Serialization with pickle

You can save the entire model (best individual, parameters, and archive) with pickle. 

> At the current stage, Brush does not serialize the search space and dataset references, but only the necessary information to be able to load a previously trained model and do predictions with it.

In [12]:
est

BrushClassifier(algorithm='nsga2', bandit='dynamic_thompson', batch_size=1.0,
                class_weights=[], cx_prob=0.14285714285714285,
                functions=['SplitBest', 'Add', 'Mul', 'Sin', 'Cos', 'Exp',
                           'Logabs'],
                initialization='uniform',
                load_population='/tmp/tmp1quqba5t/population.json', logfile='',
                max_depth=10, max_gens=10, max_size=100, max_stall=0,
                max_time=-1, mig_prob=0.05, mode...
                                'subtree': 0.16666666666666666,
                                'toggle_weight_off': 0.16666666666666666,
                                'toggle_weight_on': 0.16666666666666666},
                n_jobs=1, num_islands=5, objectives=['scorer', 'complexity'],
                pop_size=200, random_state=None, save_population='',
                scorer='average_precision_score', sel='lexicase',
                shuffle_split=False, surv='nsga2', use_arch=True,
                val_from_arch=True, validation_size=0.0, ...)

In [13]:
import pickle

est_file = os.path.join(tempfile.mkdtemp(), 'est.pkl')

with open(est_file, 'wb') as f:
    pickle.dump(est, f)

In [14]:
loaded_est = pickle.load(open(est_file, 'rb'))

In [15]:
print(est.predict(X))
print(loaded_est.predict(X))

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True False  True  True  True  True  True  True  True
  True False False False False  True False False False False False False
 False False False False False False False  True False False False False
 False False]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True False  True  True  True  True  True  True  True
  True False False False False  True False False False False False False
 False False False False False False False  True False False False False
 False False]


In [16]:
print(est.predict_archive(X)[0])
print(loaded_est.predict_archive(X)[0])

{'id': 451, 'y_pred': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False])}
{'id': 451, 'y_pred': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False])}


## Stop/resume the fitting of an estimator

In the code below I try to mimic how pytorch models are trained: we can stop the training at any time, and we can resume it later. 

The idea is to demonstrate how to use population files to store checkpoints, and continuing from the last saved checkpoint.

In [17]:
def train(est, X, y):
    checkpoint = os.path.join(tempfile.mkdtemp(), 'brush_pop_checkpoint.json')
    
    step = 5
    max_gens = est.max_gens
    est.max_gens = step
    est.save_population = checkpoint
    est.load_population = ""
    
    # You can set validation_size to a value greater than zero
    # and shuffle_split to true to have random bathes of data
    est.shuffle_split = True
    est.validation_size = 0.2
    
    for g in range(max_gens // step):
        print(f"Progress {g + 1}/{max_gens // step}")
        
        est.fit(X, y) # Notice that this will reset the MAB everytime!

        # Enable loading the checkpoint after a first run
        est.load_population = checkpoint
        
        print("Best model:", est.best_estimator_.get_model())
        print('score     :', est.score(X, y))

    # Restoring initial state
    est.max_gens = max_gens

In [18]:
est = BrushClassifier(
    objectives=["scorer", "linear_complexity"],
    scorer="balanced_accuracy",
    max_gens=50,
    validation_size=0.2,
    pop_size=100,
    max_depth=20,
    max_size=50,
    verbosity=1
)

train(est, X, y)

Progress 1/10
Completed 100% [====================]
saving final population as archive...
Saved population to file /tmp/tmphii8vd7i/brush_pop_checkpoint.json
Best model: Logistic(Sum(0.00,1.00*Tan(1.00*Add(AIDS,AIDS))))
score     : 0.54
Progress 2/10
Loaded population from /tmp/tmphii8vd7i/brush_pop_checkpoint.json of size = 200
Completed 100% [====================]
saving final population as archive...
Saved population to file /tmp/tmphii8vd7i/brush_pop_checkpoint.json
Best model: Logistic(Sum(-0.12,Sin(1.00*AIDS)))
score     : 0.56
Progress 3/10
Loaded population from /tmp/tmphii8vd7i/brush_pop_checkpoint.json of size = 200
Completed 100% [====================]
saving final population as archive...
Saved population to file /tmp/tmphii8vd7i/brush_pop_checkpoint.json
Best model: Logistic(Sum(-0.47,0.00*AIDS))
score     : 0.58
Progress 4/10
Loaded population from /tmp/tmphii8vd7i/brush_pop_checkpoint.json of size = 200
Completed 100% [====================]
saving final population as arc

By default, sklearn estimators will reset when calling fit twice. To continue from last fit, you can call `partial_fit` and brush will resume the training.

If you want, you can change parameters from the `est` object before calling partial fit to update the execution settings.

It is important that the data has the same features (same name and dtype) as the data used in the previous `fit`/`partial_fit`.

In [19]:
print(est.best_estimator_.get_model())
print(est.best_estimator_.fitness)

est.partial_fit(X, y)

print(est.best_estimator_.get_model())
print(est.best_estimator_.fitness)

Logistic(Sum(-0.53,-0.00*Sum(-251.69*Sin(1.00*Mean(0.99*AIDS,0.99*AIDS,1.00*Total)),-0.03*AIDS)))
Fitness(0.700000 81.000000 )
Loaded population from /tmp/tmphii8vd7i/brush_pop_checkpoint.json of size = 200
Completed 100% [====================]
saving final population as archive...
Saved population to file /tmp/tmphii8vd7i/brush_pop_checkpoint.json
Logistic(Sum(-0.64,If(AIDS>15890.50,1.00,If(Total>1572255.50,0.00,0.01*AIDS))))
Fitness(0.800000 44.000000 )
